# Recursion Schemes in Scala

---

## Introduction

---

Todos, de una manera u otra, hemos oido hablar de recursión. Este concepto nace de la idea de usar una definición sobre sí misma para dar lugar a un nuevo objeto. Es una idea sencilla y de enorme utilidad tanto para las matemáticas como para los lenguajes deprogramación. Lo que vamos a intentar a lo largo de este trabajito es desarrollar en  detalle esta idea, hasta el punto de abstraerla lo más posible para generar un esquema general de recursión (y de evaluación de estructuras recursivas) que podamos usar tanto para entender mejor el concepto de recursión como para aplicarlo en casos de uso reales. 

### La idea de recursión: recursión en matemáticas y en computación

Es enorme la cantidad de lugares en los que este concepto es útil, uno de ellos, en el mismo corazón del formalismo matemático, en las definiciones de números naturales (a la peano) y en la definción de sus operaciones. Es decir, entendiendo las operaciones sobre números naturales como funciones $op:\mathbb{N}\times\mathbb{N}\rightarrow\mathbb{N}$, lo que tenemos son construcciones de conjuntos "_tipos_" recursivos, que son los números naturales en este caso, y el de operaciones recursivas (suma, multiplicación, exponenciación). Veamos más en detalle este ejemplo:

    
## From expresions to recursion

Imagina que tenemos una abstracción de un anillo, es decir, un conjunto en la que existen dos operaciones, suma y preocuto y un elemento neutro para cada una de ellas, 0 y 1 respectivamente. 

In [2]:
sealed trait RingF[+A]
case object Zero extends RingF[Nothing]
case object One extends RingF[Nothing]
case class Add[A](x: A, y: A) extends RingF[A]
case class Mult[A](x:A, y: A) extends RingF[A]

defined trait RingF
defined object Zero
defined object One
defined class Add
defined class Mult

El tipo geneŕico A nos permite dejar libre las posibles implementaciones. Una vez se tiene esta estructura genérica, resulta interesante implementar una forma de interpretarla y de evaluarla, por ejemplo, podemos definir la estructura del anillo de los enteros. Una posible evaluación es a entero, de manera que implementamos las operaciones. Otra se puede hacer sobre Strings, para tener una representación de las operacioens. 

In [3]:
def evalToInt: RingF[Int] => Int = {
    x => x match {
        case Zero => 0
        case One => 1
        case Add(x, y) => x + y
        case Mult(x, y) => x*y
    }
}

def evalToStr: RingF[Int] => String = {
    x => x match {
        case Zero => "0"
        case One => "1"
        case Add(x, y) => x.toString + " + " + y.toString
        case Mult(x, y) => x.toString + " * " + y.toString
    }
}

defined function evalToInt
defined function evalToStr

Uno de los puntos a tener en cuenta es que ambas evaluaciones, esencialmente, pueden obtenerse una de la otra. Dado que tenemos una transformación Int => String, tenemos una trasformación de RingF[Int] => RingF[String]. Además, esto no se limita a esta función, este comportaminto lo podemos generar de manera abstracta a partir de cualquier función genérica A => B. Es decir, que para cada f: A => B, podemos definir una función RingF[A] => RingF[B]. Esta propiedad, a groso modo, hace que RingF sea un functor. Una implemntación de esta abstracción puede ser 

In [4]:
sealed trait Functor[F[_]] {
    def map[A,B](f: A => B): F[A] => F[B]
}

implicit val RingFunctor = new Functor[RingF] {
    override def map[A, B](f: A => B): RingF[A] => RingF[B] = {
        x => x match {
            case Zero => Zero
            case One => One
            case Add(x, y) => Add(f(x), f(y))
            case Mult(x, y) => Mult(f(x), f(y))
        }
    }
}

defined trait Functor
RingFunctor: AnyRef with Functor[RingF] = ammonite.$sess.cmd3$Helper$$anon$1@2e5bd674

Una vez que tenemos esta abstracción, podemos reimplementar el evalToStrig a partir del método map

Hasta aquí todo genial, sin embargo, lo normal es que nos sepa a poco el definir operaciones binarioas de la forma 1 + 2. Si queremos definir la expresión 1 + 2x3 necesitamos cambiar la estructura a algo de la forma 


In [4]:
val z: RingF[RingF[Int]] = Add(One, Mult(2, 3))

z: RingF[RingF[Int]] = Add(One,Mult(2,3))


Cada vez que queramos generar una expresión más complicada, necesitamos, por un lado, añadir un tipo RingF al constructor y, además, redefinir el método evaltoInt. Esto da lugar a una estructura recursiva sobre las expresiones que generan las operaciones del anillo y, además, una dependencia en la forma de evaluarlo. Una de las posibles soluciones, y la más directa, es usar una extensión recursiva sobre RingF de manera que tengamos

In [4]:
sealed trait DirectRingF 
case object Zero extends DirectRingF
case object One extends DirectRingF
case class Add(x: DirectRingF, y: DirectRingF) extends DirectRingF
case class Mult(x: DirectRingF, y: DirectRingF) extends DirectRingF

def directEval: DirectRingF => Int = {
    x => x match {
        case Zero => 0
        case One => 1
        case Add(x, y) => directEval(x) + directEval(y)
        case Mult(x, y) => directEval(x) * directEval(y)
    }
}

defined trait DirectRingF
defined object Zero
defined object One
defined class Add
defined class Mult
defined function directEval

Esta sintaxis es perfectamente correcta. Sin embargo, se pierde queda un poco oscuro el paso de la evaluación de las expresiones de los nodos y las de las hojas. Por ello, y para disfrutar rompiéndonos un poco la cabeza, vamos a tratar de pensar en lo qeu significa la estructura recursiva en este caso.

En primer lugar, tenemos un functor, es decir, un tipo que come otros tipos para generar otros nuevos. en nuestro caso, es de la forma

A => F[A] => F[F[A]] => ...

Además, si conocemos una evaluación concreta, por ejemplo, de tipo Int, tenemos una función F[Int] => Int. Es hora de empezar a dar nombre a esta estructura, es lo que se llama una F-álgebra. Más en concreto, dado un functor F una F-álgebra es un par (f, A), donde A es un tipo concreto y f es una aplicaión F[A] => A.

Por ejemplo, (evalToInt, Int) es una RingF-álgebra. Básicamente, las F-álgebras codifican la manera en la que podemos evaluar expresiones generadas a partir del functor dado. 

Por otro lado, si tratamos de imaginar la idea de recursión para un cierto functor F, es como el resultado de aplicar F una cantidad infinita de veces, de manera que siempre pudiesemos cosntrur cualquier expresión a partir de la composición de objetos de tipo RingF[RingF[...]]. Es decir, el tipo recursivo, H,  para RingF debe ser un tipo con la propiedad de que RingF[H] = H, es decir, el final de la cadena de composiciones del functor RingF. A partir de la ecuación anterior, al tipo recursivo asociado a RingF se le conoce como punto fijo. Esta idea nos permite abstraer la noción de recursión y codificarla en la idea de los puntos fijos. Una implementación sencilla de un punto fijo genérico para un functor es el siguiente.

In [5]:
case class Fix[F[_]](unfix: F[Fix[F]] )
object Fix {
  def in[F[_]](ff: F[Fix[F]]): Fix[F] = new Fix[F](ff)
  def out[F[_]]: Fix[F] => F[Fix[F]] = f => f.unfix
}

defined class Fix
defined object Fix

Las funciones in y out son las que representan la igualdad de tipos ya que podemos ir de un tipo a otro sin perder nada de información (es un isomorfismo). A partir de este tipo, podemos contruir nuestros tipo recursivo asociado al functo RingF de la manera siguiente

In [6]:
type Ring = Fix[RingF]
val zero = Fix[RingF](Zero)
val one =  Fix[RingF](One)
def add: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Add(x, y))
def mult: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Mult(x, y))

defined type Ring
zero: Fix[RingF] = Fix(Zero)
one: Fix[RingF] = Fix(One)
defined function add
defined function mult

Con esto, ya podemos definir expresiones tan complejas ocmo deseemos:

In [8]:
val exp = add(one, mult(add(one, one), one))

exp: Ring = Fix(
  Add(Fix(One), Fix(Mult(Fix(Add(Fix(One), Fix(One))), Fix(One))))
)

Uno de los objetivos lo hemos conseguido, en realidad, el trait Fix nos permite construir la estructura recursiva de cualquier functor, es decir, que dada cualquier definición de tipos sobre el caso base, podemos levantarla a su estructura recursiva. Sin embargo, esto no es muy interesanete si no podemos hacer lo mismo con las evaluaciones. Ya hemos comentado que las evaluaciones se basan en F-álgebras. 

In [7]:
type Algebra[F[_], A] = F[A] => A

defined type Algebra

A partir de este punto, nos gustaria deducir cómo funciona la evaluación de las estructuras recursivas a partir de un álgebra concreta. Es aquí donde entra en juego de manera crucial que F sea un functor y que, por tanto, pueda generar aplicaciones entre las tipos contruidos con F. Sea i el punto fijo del functor F, tenemos que F[i] = i ( en el sentido de las aplicaciones, es decir que:

F[i] -----> F[A]
 |             |
 |             |
 i ---------> A
 
 tenemos bien definidas tanto la aplicación in, como out, como el evaluador. Podemos contruir las dos funciones que conectan el álgebra del punto fijo con el ágebra dada. La respuesta es que sí, y esta función no depende del álgebra inicial: 

In [8]:
def cata[F[_], A](alg: Algebra[F, A])(implicit F: Functor[F]): Fix[F] => A = {
    x => alg((F.map(cata(alg))(Fix.out(x))))
}

defined function cata

Con esta función, ya podemos generar la versión recursiva de evalToInt

In [9]:
def evalToInt: Algebra[RingF, Int] = {
    x => x match {
        case Zero => 0
        case One => 1
        case Add(x, y) => x + y
        case Mult(x, y) => x*y
    }
}
cata(evalToInt)(RingFunctor)(exp)

defined function evalToInt
res8_1: Int = 3

## Recursive data Types

Uno de los ejemplos más claros de tipos recursivos es el de litas. Una implementación sencilla sería 

---

In [12]:
def evalPol: Algebra[RingF, List[Int]] = {
    x => x match {
        case Zero => 0::Nil
        case One => 1::Nil
        case Add(x, y) => {
            val max =  if (x.length >= y.length) x else y
            val min = if (x.length >= y.length) y else x
            val N = max.length - min.length
            val nmin= expand0(N, min, 0)
            max.zip(min).map(x => x match {
                case (i, j) => i + j
            })
        }
        case Mult(x, y) => {
            val z = expand0(x.length + y.length, Nil)
            (0 to (x.length + y.length)).map(i => y.map(j => i*j))
        }
    }
}

cmd12.sc:9: too many arguments (3) for method expand0: (n: Int, x: List[Int])List[Int]
            val nmin= expand0(N, min, 0)
                                      ^cmd12.sc:16: type mismatch;
 found   : scala.collection.immutable.IndexedSeq[List[Int]]
 required: List[Int]
            (0 to (x.length + y.length)).map(i => y.map(j => i*j))
                                            ^Compilation Failed

: 

In [12]:
import annotation.tailrec
val x = 1::2::3::Nil
val y = 3::2::Nil
val N = Integer.max(x.length,  y.length)
@tailrec
def expand(n: Int, x: List[Int], i: Int): List[Int] = {
    val z = x
    n match {
        case 0 =>  z
        case _ => expand(n-1, i::x, i)
    }
}
x(0)

import annotation.tailrec

x: List[Int] = List(1, 2, 3)
y: List[Int] = List(3, 2)
N: Int = 3
defined function expand
res11_5: Int = 1

In [4]:
sealed trait myList[+A]
case object Nill extends myList[Nothing]
case class Cons[A](h: A, t: myList[A]) extends myList[A]

defined trait myList
defined object Nill
defined class Cons

Como puede verse, tenemos un caso base que es la lista vacía y un elemento Cons que representa la lista formada por una cabecera y su cola. 

In [22]:
trait Functor[F[_]] {
    def map[A, B](f: A => B): F[A] => F[B]
}

sealed trait ExprF[A] 
final case class c[A](x: Double) extends ExprF[A] 
final case class  v[A](x: String) extends ExprF[A]
final case class  add[A](x: A, y: A) extends ExprF[A]
final case class  times[A](x: A, y : A) extends ExprF[A]
val exprfunctor = new Functor[ExprF] {
    override def map[A, B](f: A => B): ExprF[A] => ExprF[B] = {
        exp1 => exp1 match {
            case c(x) => c(x)
            case v(s) => v(s)
            case add(a, b) => add(f(a), f(b))
            case times(a, b) => times(f(a), f(b))
        }
    }
}

defined trait Functor
defined trait ExprF
defined class c
defined class v
defined class add
defined class times
exprfunctor: AnyRef with Functor[ExprF] = ammonite.$sess.cmd21$Helper$$anon$1@50432c7f

In [23]:
type Algebra[F[_], A] = F[A] => A

defined type Algebra

In [24]:
case class Assing(value: Double, variable: String)

def evalStr: Algebra[ExprF, String] = {
x => x match {
    case c(x) => x.toString
    case v(x) => x
    case add(a, b) => a + " + " +  b
    case times(a, b) => "(" + a + ")" + "*" +"(" + b + ")"
    }
}

def evalDouble(y: Assing*): Algebra[ExprF, Double] = {
    x => x match {
        case c(x) => x
        case v(x) => {
            val value = y.filter(z => z.variable == x)
            if (value.length > 0) value(0).value else 0
    }
        case add(a, b) => a + b
        case times(a, b) => a * b
}
}

defined class Assing
defined function evalStr
defined function evalDouble

In [25]:
case class Fix[F[_]](unfix: F[Fix[F]] )
object Fix {
  def in[F[_]](ff: F[Fix[F]]): Fix[F] = new Fix[F](ff)
  def out[F[_]]: Fix[F] => F[Fix[F]] = f => f.unfix
}

defined class Fix
defined object Fix

In [26]:
type Expr = Fix[ExprF]
def Var: String => Expr = s => Fix[ExprF](v(s))
def Const: Double => Expr = x => Fix[ExprF](c(x))
def Add: (Expr, Expr) => Expr = (a, b) => Fix[ExprF](add(a, b))
def Times: (Expr, Expr) => Expr = (a, b) => Fix[ExprF](times(a, b))


defined type Expr
defined function Var
defined function Const
defined function Add
defined function Times

In [27]:
val x = Add(Add(Var("x"), Const(7)), Add(Var("x"), Const(7)))

x: Expr = Fix(
  add(Fix(add(Fix(v("x")), Fix(c(7.0)))), Fix(add(Fix(v("x")), Fix(c(7.0)))))
)

In [27]:
zero: Nat = Fix(Zero)

(console):1:11 expected (Semis | &"}" | end-of-input)
zero: Nat = Fix(Zero)
          ^

: 

In [28]:
sealed trait NatF[+A]
case object Zero extends NatF[Nothing]
case class Succ[A](x: A) extends NatF[A]

val NatFunctor = new Functor[NatF] {
    override def map[A,B](f: A => B): NatF[A] => NatF[B] = {
        case Zero => Zero
        case Succ(x) => Succ(f(x))
    }
}

type Nat = Fix[NatF]
val zero: Nat =  Fix[NatF](Zero)
def suc: Nat => Nat = x =>  Fix[NatF](Succ(x))

defined trait NatF
defined object Zero
defined class Succ
NatFunctor: AnyRef with Functor[NatF] = ammonite.$sess.cmd27$Helper$$anon$1@5601c43e
defined type Nat
zero: Nat = Fix(Zero)
defined function suc

In [29]:
val num = suc(suc(zero))

num: Nat = Fix(Succ(Fix(Succ(Fix(Zero)))))

In [30]:
def evalToInt: Algebra[NatF, Int] = {
    x => x match {
        case Zero => 0
        case Succ(x) => x + 1 
    }
}
def evaltoFib: Algebra[NatF, (Int, Int)] = {
    case Zero => (1, 1)
    case Succ((n, m)) => (m, n+m) 
}


defined function evalToInt
defined function evaltoFib

In [30]:
cata(evalStr)(exprfunctor)(x)

cmd30.sc:1: type mismatch;
 found   : ammonite.$sess.cmd23.wrapper.cmd22.Algebra[ammonite.$sess.cmd23.wrapper.cmd21.ExprF,String]
    (which expands to)  ammonite.$sess.cmd23.wrapper.cmd21.ExprF[String] => String
 required: ammonite.$sess.cmd18.wrapper.cmd16.Algebra[F,String]
    (which expands to)  F[String] => String
val res30 = cata(evalStr)(exprfunctor)(x)
                 ^Compilation Failed

: 

In [30]:
cata(evalDouble(Assing(3, "x")))(exprfunctor)(x)

cmd30.sc:1: type mismatch;
 found   : ammonite.$sess.cmd23.wrapper.cmd22.Algebra[ammonite.$sess.cmd23.wrapper.cmd21.ExprF,Double]
    (which expands to)  ammonite.$sess.cmd23.wrapper.cmd21.ExprF[Double] => Double
 required: ammonite.$sess.cmd18.wrapper.cmd16.Algebra[F,Double]
    (which expands to)  F[Double] => Double
val res30 = cata(evalDouble(Assing(3, "x")))(exprfunctor)(x)
                           ^Compilation Failed

: 

In [30]:
cata(evaltoFib)(NatFunctor)(suc(num))

cmd30.sc:1: type mismatch;
 found   : ammonite.$sess.cmd29.wrapper.cmd22.Algebra[ammonite.$sess.cmd29.wrapper.cmd27.NatF,(Int, Int)]
    (which expands to)  ammonite.$sess.cmd29.wrapper.cmd27.NatF[(Int, Int)] => (Int, Int)
 required: ammonite.$sess.cmd18.wrapper.cmd16.Algebra[F,(Int, Int)]
    (which expands to)  F[(Int, Int)] => (Int, Int)
val res30 = cata(evaltoFib)(NatFunctor)(suc(num))
                 ^Compilation Failed

: 

In [45]:
cata(evalToInt)(NatFunctor)(num)

res3: Int = 2


In [3]:
%load_ext itikz

<console>: 26: error: not found: value %

In [4]:
%%itikz
\documentclass{standalone}
\usepackage{tikz-cd}
\usepackage{adjustbox}
\begin{document}
\adjustbox{scale=3,center}{%
    \begin{tikzcd}
    X \arrow[r, hook] \arrow[dr, dashrightarrow]
    & \bar{X} \arrow[d]\\
    & Y
    \end{tikzcd}
}
\end{document}

<console>: 4: error: error in unicode escape